In [2]:
from functions import *
import pandas as pd
import os


In [1]:
file_path = os.environ.get("FILE_PATH", "./prompt_library.xlsx")
sheet_name = os.environ.get("SHEET_NAME", "categorized")
dataset = pd.read_excel(file_path,sheet_name=sheet_name)


NameError: name 'pd' is not defined

In [4]:
limit = 5 #no. of questions to prompt
dataset.head(limit)

In [7]:
# Process each row and store the results in a new DataFrame
results = []
for index, row in dataset.iterrows():
    try:
        if index >= limit:
            break
        prompt = row["Prompt"]
        answer, response_time = process_prompt(prompt)

        accuracy = evaluate_response(prompt, answer, EvaluationType.ACCURACY)
        relevance = evaluate_response(prompt, answer, EvaluationType.RELEVANCE)
        results.append((prompt,answer,response_time,accuracy,relevance))

        print(f"Question {index} processed succesfully")
    except Exception as e:
        print(f"Error Question {index} processed unsuccesfully: {e}")
        continue

Question 0 processed succesfully
Question 1 processed succesfully
Question 2 processed succesfully
Question 3 processed succesfully
Question 4 processed succesfully


In [5]:
# Create a new DataFrame from the results
output_dataset = pd.DataFrame(results, columns=['Question', 'Answer', 'Response Time (s)', 'Accuracy','Relevance'])

In [6]:
# Write the new DataFrame to a new Excel file
output_file_path = 'output_data.xlsx'
output_dataset.to_excel(output_file_path, index=False)

In [7]:
output_dataset

,Question,Answer,Response Time (s),Accuracy,Relevance
0,Can you identify the main AI and machine learn...,The documents provided highlight several AI an...,86.714674,I would rate the output a 4.\n\nThe response i...,I would rate the output a 4.\n\nThe response i...
1,How did Microsoft researchers address issues o...,Microsoft researchers addressed issues of bias...,84.063624,The response provided is accurate in summarizi...,I would rate the response a 4.\n\nThe response...
2,How did Microsoft's research at NeurIPS 2023 c...,Microsoft's research at NeurIPS 2023 contribut...,61.018703,I would rate the output a 4.\n\nThe response i...,I would rate the relevance of the output as a ...
3,How do Microsoft's papers at NeurIPS contribut...,The requested information is not available in ...,18.117342,The output is accurate. The user's question is...,1
4,How do Microsoft's NeurIPS 2023 submissions ad...,Microsoft's submissions to NeurIPS 2023 showca...,72.670112,I would rate the output a 4.\n\nThe response i...,I would rate the relevance of the output to th...


In [8]:
print(f"Processing complete. Results saved to {output_file_path}")

Processing complete. Results saved to output_data.xlsx
